In [7]:
!pip install flask flask_sqlalchemy pymysql


In [3]:
pip show flask


Name: Flask
Version: 3.0.3
Summary: A simple framework for building complex web applications.
Home-page: 
Author: 
Author-email: 
License: 
Location: C:\Users\chenm\anaconda3\Lib\site-packages
Requires: blinker, click, itsdangerous, Jinja2, Werkzeug
Required-by: Flask-SQLAlchemy
Note: you may need to restart the kernel to use updated packages.


In [3]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///restaurant_system.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# --------------------- Student ---------------------
class Student(db.Model):
    __tablename__ = 'student'
    student_id = db.Column(db.Integer, primary_key=True)
    student_name = db.Column(db.String(100), nullable=False)
    password = db.Column(db.String(100), nullable=False)

    orders = db.relationship('Order', back_populates='student')
    browses = db.relationship('Browse', back_populates='student')
    comments = db.relationship('Comment', back_populates='student')
    carts = db.relationship('Cart', back_populates='student')

# --------------------- Restaurant ---------------------
class Restaurant(db.Model):
    __tablename__ = 'restaurant'
    restaurant_id = db.Column(db.Integer, primary_key=True)
    restaurant_name = db.Column(db.String(100), nullable=False)
    category = db.Column(db.String(100))
    address = db.Column(db.String(200))
    opening_hour = db.Column(db.String(100))

    orders = db.relationship('Order', back_populates='restaurant')
    browses = db.relationship('Browse', back_populates='restaurant')
    comments = db.relationship('Comment', back_populates='restaurant')
    coupons = db.relationship('Coupon', back_populates='restaurant')
    menus = db.relationship('Menu', back_populates='restaurant')
    ads = db.relationship('Advertisement', back_populates='restaurant')

# --------------------- Cart ---------------------
class Cart(db.Model):
    __tablename__ = 'cart'
    cart_id = db.Column(db.Integer, primary_key=True)
    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'))

    student = db.relationship('Student', back_populates='carts')
    cart_items = db.relationship('CartItem', back_populates='cart')

# --------------------- CartItem ---------------------
class CartItem(db.Model):
    __tablename__ = 'cart_item'
    cart_item_id = db.Column(db.Integer, primary_key=True)
    quantity = db.Column(db.Integer, default=1)

    cart_id = db.Column(db.Integer, db.ForeignKey('cart.cart_id'))
    menu_item_id = db.Column(db.Integer, db.ForeignKey('menu_item.menu_item_id'))

    cart = db.relationship('Cart', back_populates='cart_items')
    menu_item = db.relationship('MenuItem')

# --------------------- Order ---------------------
class Order(db.Model):
    __tablename__ = 'order'
    order_id = db.Column(db.Integer, primary_key=True)
    order_time = db.Column(db.DateTime, default=datetime.utcnow)
    order_price = db.Column(db.Float, default=0.0)

    items = db.relationship('OrderItem', back_populates='order')
    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    student = db.relationship('Student', back_populates='orders')
    restaurant = db.relationship('Restaurant', back_populates='orders')

# --------------------- OrderItem ---------------------
class OrderItem(db.Model):
    __tablename__ = 'order_item'
    order_item_id = db.Column(db.Integer, primary_key=True)
    quantity = db.Column(db.Integer, default=1)
    price = db.Column(db.Float, nullable=False)

    order_id = db.Column(db.Integer, db.ForeignKey('order.order_id'))
    menu_item_id = db.Column(db.Integer, db.ForeignKey('menu_item.menu_item_id'))

    order = db.relationship('Order', back_populates='items')
    menu_item = db.relationship('MenuItem')

# --------------------- Menu ---------------------
class Menu(db.Model):
    __tablename__ = 'menu'
    menu_id = db.Column(db.Integer, primary_key=True)
    average_price = db.Column(db.Float)

    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    restaurant = db.relationship('Restaurant', back_populates='menus')
    menu_items = db.relationship('MenuItem', back_populates='menu')

# --------------------- MenuItem ---------------------
class MenuItem(db.Model):
    __tablename__ = 'menu_item'
    menu_item_id = db.Column(db.Integer, primary_key=True)
    item_name = db.Column(db.String(100), nullable=False)
    price = db.Column(db.Float, nullable=False)
    meal_name = db.Column(db.String(100))

    menu_id = db.Column(db.Integer, db.ForeignKey('menu.menu_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))

    menu = db.relationship('Menu', back_populates='menu_items')

# --------------------- Browse ---------------------
class Browse(db.Model):
    __tablename__ = 'browse'
    browse_id = db.Column(db.Integer, primary_key=True)
    browse_time = db.Column(db.DateTime, default=datetime.utcnow)

    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))

    student = db.relationship('Student', back_populates='browses')
    restaurant = db.relationship('Restaurant', back_populates='browses')

# --------------------- Comment ---------------------
class Comment(db.Model):
    __tablename__ = 'comment'
    comment_id = db.Column(db.Integer, primary_key=True)
    comment_level = db.Column(db.Integer)
    comment_text = db.Column(db.Text, nullable=False)
    respond = db.Column(db.Text, nullable=True)

    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))

    student = db.relationship('Student', back_populates='comments')
    restaurant = db.relationship('Restaurant', back_populates='comments')

# --------------------- Coupon ---------------------
class Coupon(db.Model):
    __tablename__ = 'coupon'
    coupon_id = db.Column(db.Integer, primary_key=True)
    time = db.Column(db.DateTime, default=datetime.utcnow)
    coupon_offer = db.Column(db.String(255))

    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    restaurant = db.relationship('Restaurant', back_populates='coupons')

# --------------------- Advertisement ---------------------
class Advertisement(db.Model):
    __tablename__ = 'advertisement'
    advertisement_id = db.Column(db.Integer, primary_key=True)
    ad_start_time = db.Column(db.DateTime)
    ad_end_time = db.Column(db.DateTime)
    ad_watch_time = db.Column(db.Integer)
    ad_url = db.Column(db.String(255))

    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    restaurant = db.relationship('Restaurant', back_populates='ads')

# --------------------- 初始化資料庫 ---------------------
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        print("✅ 資料庫與資料表建立完成！")
    app.run(debug=True)



✅ 資料庫與資料表建立完成！
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\chenm\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///restaurant_system.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# --------------------- Cart CRUD ---------------------
@app.route("/cart", methods=["POST"])
def create_cart():
    data = request.get_json()
    sql = text("""
        INSERT INTO cart (student_id)
        VALUES (:student_id)
    """)
    db.session.execute(sql, {"student_id": data.get("student_id")})
    db.session.commit()
    return jsonify({"message": "✅ 購物車新增成功"})

@app.route("/carts", methods=["GET"])
def get_all_carts():
    sql = text("SELECT * FROM cart")
    result = db.session.execute(sql)
    carts = [dict(row) for row in result.mappings()]
    return jsonify(carts)
def update_cart(cart_id):
    data = request.get_json()
    sql = text("UPDATE cart SET student_id = :student_id WHERE cart_id = :id")
    db.session.execute(sql, {"student_id": data.get("student_id"), "id": cart_id})
    db.session.commit()
    return jsonify({"message": "✅ 購物車已更新"})
    
@app.route("/cart/<int:cart_id>", methods=["DELETE"])
def delete_cart(cart_id):
    sql = text("DELETE FROM cart WHERE cart_id = :id")
    db.session.execute(sql, {"id": cart_id})
    db.session.commit()
    return jsonify({"message": "✅ 購物車已刪除"})

# --------------------- CartItem CRUD ---------------------
@app.route("/cart_item", methods=["POST"])
def create_cart_item():
    data = request.get_json()
    sql = text("""
        INSERT INTO cart_item (cart_id, menu_item_id, quantity)
        VALUES (:cart_id, :menu_item_id, :quantity)
    """)
    db.session.execute(sql, {
        "cart_id": data.get("cart_id"),
        "menu_item_id": data.get("menu_item_id"),
        "quantity": data.get("quantity")
    })
    db.session.commit()
    return jsonify({"message": "✅ 購物車項目新增成功"})

@app.route("/cart_items", methods=["GET"])
def get_all_cart_items():
    sql = text("SELECT * FROM cart_item")
    result = db.session.execute(sql)
    items = [dict(row) for row in result.mappings()]
    return jsonify(items)

@app.route("/cart_item/<int:cart_item_id>", methods=["PUT"])
def update_cart_item(cart_item_id):
    data = request.get_json()
    sql = text("""
        UPDATE cart_item
        SET cart_id = :cart_id,
            menu_item_id = :menu_item_id,
            quantity = :quantity
        WHERE cart_item_id = :id
    """)
    db.session.execute(sql, {
        "cart_id": data.get("cart_id"),
        "menu_item_id": data.get("menu_item_id"),
        "quantity": data.get("quantity"),
        "id": cart_item_id
    })
    db.session.commit()
    return jsonify({"message": "✅ 購物車項目已更新"})

@app.route("/cart_item/<int:cart_item_id>", methods=["DELETE"])
def delete_cart_item(cart_item_id):
    sql = text("DELETE FROM cart_item WHERE cart_item_id = :id")
    db.session.execute(sql, {"id": cart_item_id})
    db.session.commit()
    return jsonify({"message": "✅ 購物車項目已刪除"})

# --------------------- MenuItem CRUD ---------------------
@app.route("/menu_item", methods=["POST"])
def create_menu_item():
    data = request.get_json()
    sql = text("""
        INSERT INTO menu_item (item_name, price, meal_name, menu_id, restaurant_id)
        VALUES (:item_name, :price, :meal_name, :menu_id, :restaurant_id)
    """)
    db.session.execute(sql, {
        "item_name": data.get("item_name"),
        "price": data.get("price"),
        "meal_name": data.get("meal_name"),
        "menu_id": data.get("menu_id"),
        "restaurant_id": data.get("restaurant_id")
    })
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目新增成功"})

@app.route("/menu_items", methods=["GET"])
def get_all_menu_items():
    sql = text("SELECT * FROM menu_item")
    result = db.session.execute(sql)
    items = [dict(row) for row in result.mappings()]
    return jsonify(items)

@app.route("/menu_item/<int:menu_item_id>", methods=["PUT"])
def update_menu_item(menu_item_id):
    data = request.get_json()
    sql = text("""
        UPDATE menu_item
        SET item_name = :item_name,
            price = :price,
            meal_name = :meal_name,
            menu_id = :menu_id,
            restaurant_id = :restaurant_id
        WHERE menu_item_id = :id
    """)
    db.session.execute(sql, {
        "item_name": data.get("item_name"),
        "price": data.get("price"),
        "meal_name": data.get("meal_name"),
        "menu_id": data.get("menu_id"),
        "restaurant_id": data.get("restaurant_id"),
        "id": menu_item_id
    })
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目已更新"})

@app.route("/menu_item/<int:menu_item_id>", methods=["DELETE"])
def delete_menu_item(menu_item_id):
    sql = text("DELETE FROM menu_item WHERE menu_item_id = :id")
    db.session.execute(sql, {"id": menu_item_id})
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目已刪除"})
# --------------------- Menu CRUD ---------------------
@app.route("/menu", methods=["POST"])
def create_menu():
    data = request.get_json()
    sql = text("INSERT INTO menu (average_price, restaurant_id) VALUES (:average_price, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 菜單新增成功"})

@app.route("/menus", methods=["GET"])
def get_menus():
    sql = text("SELECT * FROM menu")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/menu/<int:id>", methods=["GET"])
def get_menu(id):
    sql = text("SELECT * FROM menu WHERE menu_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到菜單"}), 404)

@app.route("/menu/<int:id>", methods=["PUT"])
def update_menu(id):
    data = request.get_json()
    sql = text("UPDATE menu SET average_price = :average_price, restaurant_id = :restaurant_id WHERE menu_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 菜單更新成功"})

@app.route("/menu/<int:id>", methods=["DELETE"])
def delete_menu(id):
    sql = text("DELETE FROM menu WHERE menu_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 菜單刪除成功"})

# --------------------- Browse CRUD ---------------------
@app.route("/browse", methods=["POST"])
def create_browse():
    data = request.get_json()
    sql = text("INSERT INTO browse (browse_time, student_id, restaurant_id) VALUES (:browse_time, :student_id, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 瀏覽紀錄新增成功"})

@app.route("/browses", methods=["GET"])
def get_browses():
    sql = text("SELECT * FROM browse")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/browse/<int:id>", methods=["GET"])
def get_browse(id):
    sql = text("SELECT * FROM browse WHERE browse_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到瀏覽紀錄"}), 404)

@app.route("/browse/<int:id>", methods=["DELETE"])
def delete_browse(id):
    sql = text("DELETE FROM browse WHERE browse_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 瀏覽紀錄刪除成功"})

# --------------------- Comment CRUD ---------------------
@app.route("/comment", methods=["POST"])
def create_comment():
    data = request.get_json()
    sql = text("INSERT INTO comment (comment_level, comment_text, respond, parent_id, student_id, restaurant_id) VALUES (:comment_level, :comment_text, :respond, :parent_id, :student_id, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 留言新增成功"})

@app.route("/comments", methods=["GET"])
def get_comments():
    sql = text("SELECT * FROM comment")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/comment/<int:id>", methods=["GET"])
def get_comment(id):
    sql = text("SELECT * FROM comment WHERE comment_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到留言"}), 404)

@app.route("/comment/<int:id>", methods=["PUT"])
def update_comment(id):
    data = request.get_json()
    sql = text("UPDATE comment SET comment_level = :comment_level, comment_text = :comment_text, respond = :respond, parent_id = :parent_id, student_id = :student_id, restaurant_id = :restaurant_id WHERE comment_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 留言更新成功"})

@app.route("/comment/<int:id>", methods=["DELETE"])
def delete_comment(id):
    sql = text("DELETE FROM comment WHERE comment_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 留言刪除成功"})

# --------------------- Coupon CRUD ---------------------
@app.route("/coupon", methods=["POST"])
def create_coupon():
    data = request.get_json()
    sql = text("INSERT INTO coupon (time, restaurant_id) VALUES (:time, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 優惠券新增成功"})

@app.route("/coupons", methods=["GET"])
def get_coupons():
    sql = text("SELECT * FROM coupon")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/coupon/<int:id>", methods=["GET"])
def get_coupon(id):
    sql = text("SELECT * FROM coupon WHERE coupon_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到優惠券"}), 404)

@app.route("/coupon/<int:id>", methods=["PUT"])
def update_coupon(id):
    data = request.get_json()
    sql = text("UPDATE coupon SET time = :time, restaurant_id = :restaurant_id WHERE coupon_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 優惠券更新成功"})

@app.route("/coupon/<int:id>", methods=["DELETE"])
def delete_coupon(id):
    sql = text("DELETE FROM coupon WHERE coupon_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 優惠券刪除成功"})

# --------------------- Advertisement CRUD ---------------------
@app.route("/advertisement", methods=["POST"])
def create_advertisement():
    data = request.get_json()
    sql = text("INSERT INTO advertisement (ad_start_time, ad_end_time, ad_watch_time, ad_url, restaurant_id) VALUES (:ad_start_time, :ad_end_time, :ad_watch_time, :ad_url, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 廣告新增成功"})

@app.route("/advertisements", methods=["GET"])
def get_advertisements():
    sql = text("SELECT * FROM advertisement")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/advertisement/<int:id>", methods=["GET"])
def get_advertisement(id):
    sql = text("SELECT * FROM advertisement WHERE advertisement_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到廣告"}), 404)

@app.route("/advertisement/<int:id>", methods=["PUT"])
def update_advertisement(id):
    data = request.get_json()
    sql = text("UPDATE advertisement SET ad_start_time = :ad_start_time, ad_end_time = :ad_end_time, ad_watch_time = :ad_watch_time, ad_url = :ad_url, restaurant_id = :restaurant_id WHERE advertisement_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 廣告更新成功"})

@app.route("/advertisement/<int:id>", methods=["DELETE"])
def delete_advertisement(id):
    sql = text("DELETE FROM advertisement WHERE advertisement_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 廣告刪除成功"})
# --------------------- Student CRUD ---------------------
@app.route("/student", methods=["POST"])
def create_student():
    data = request.get_json()
    sql = text("INSERT INTO student (student_name, password) VALUES (:student_name, :password)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 學生新增成功"})

@app.route("/students", methods=["GET"])
def get_students():
    sql = text("SELECT * FROM student")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/student/<int:id>", methods=["GET"])
def get_student(id):
    sql = text("SELECT * FROM student WHERE student_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到學生"}), 404)

@app.route("/student/<int:id>", methods=["PUT"])
def update_student(id):
    data = request.get_json()
    sql = text("UPDATE student SET student_name = :student_name, password = :password WHERE student_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 學生更新成功"})

@app.route("/student/<int:id>", methods=["DELETE"])
def delete_student(id):
    sql = text("DELETE FROM student WHERE student_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 學生刪除成功"})

# --------------------- Restaurant CRUD ---------------------
@app.route("/restaurant", methods=["POST"])
def create_restaurant():
    data = request.get_json()
    sql = text("INSERT INTO restaurant (restaurant_name, category, address, opening_hour) VALUES (:restaurant_name, :category, :address, :opening_hour)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 餐廳新增成功"})

@app.route("/restaurants", methods=["GET"])
def get_restaurants():
    sql = text("SELECT * FROM restaurant")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/restaurant/<int:id>", methods=["GET"])
def get_restaurant(id):
    sql = text("SELECT * FROM restaurant WHERE restaurant_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到餐廳"}), 404)

@app.route("/restaurant/<int:id>", methods=["PUT"])
def update_restaurant(id):
    data = request.get_json()
    sql = text("UPDATE restaurant SET restaurant_name = :restaurant_name, category = :category, address = :address, opening_hour = :opening_hour WHERE restaurant_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 餐廳更新成功"})

@app.route("/restaurant/<int:id>", methods=["DELETE"])
def delete_restaurant(id):
    sql = text("DELETE FROM restaurant WHERE restaurant_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 餐廳刪除成功"})

# --------------------- Order CRUD ---------------------
@app.route("/order", methods=["POST"])
def create_order():
    data = request.get_json()
    sql = text("INSERT INTO 'order' (order_time, order_price, student_id, restaurant_id) VALUES (:order_time, :order_price, :student_id, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 訂單新增成功"})

@app.route("/orders", methods=["GET"])
def get_orders():
    sql = text("SELECT * FROM 'order'")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/order/<int:id>", methods=["GET"])
def get_order(id):
    sql = text("SELECT * FROM 'order' WHERE order_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到訂單"}), 404)

@app.route("/order/<int:id>", methods=["PUT"])
def update_order(id):
    data = request.get_json()
    sql = text("UPDATE 'order' SET order_time = :order_time, order_price = :order_price, student_id = :student_id, restaurant_id = :restaurant_id WHERE order_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單更新成功"})

@app.route("/order/<int:id>", methods=["DELETE"])
def delete_order(id):
    sql = text("DELETE FROM 'order' WHERE order_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單刪除成功"})

# --------------------- OrderItem CRUD ---------------------
@app.route("/order_item", methods=["POST"])
def create_order_item():
    data = request.get_json()
    sql = text("INSERT INTO order_item (meal_name, quantity, price, order_id) VALUES (:meal_name, :quantity, :price, :order_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 訂單明細新增成功"})

@app.route("/order_items", methods=["GET"])
def get_order_items():
    sql = text("SELECT * FROM order_item")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/order_item/<int:id>", methods=["GET"])
def get_order_item(id):
    sql = text("SELECT * FROM order_item WHERE id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到訂單明細"}), 404)

@app.route("/order_item/<int:id>", methods=["PUT"])
def update_order_item(id):
    data = request.get_json()
    sql = text("UPDATE order_item SET meal_name = :meal_name, quantity = :quantity, price = :price, order_id = :order_id WHERE id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單明細更新成功"})

@app.route("/order_item/<int:id>", methods=["DELETE"])
def delete_order_item(id):
    sql = text("DELETE FROM order_item WHERE id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單明細刪除成功"})





# --------------------- 初始化資料庫 ---------------------
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        print("✅ 資料庫與資料表建立完成！")
    app.run(debug=True)



✅ 資料庫與資料表建立完成！
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\chenm\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
